In [29]:
!pip install --upgrade datasets

^C

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
ERROR: Operation cancelled by user


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# sts benchmark

## Download

In [62]:
from datasets import load_dataset

ds = load_dataset("mteb/stsbenchmark-sts")

ds['train'].to_csv('./datasets/raw/sts-train.csv')
ds['test'].to_csv('./datasets/raw/sts-test.csv')
ds['validation'].to_csv('./datasets/raw/sts-val.csv')

Creating CSV from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 66.09ba/s]


259728

## Transform

In [63]:
for dataset_type in ['train', 'test', 'val']:
    df = pd.read_csv(f'./datasets/raw/sts-{dataset_type}.csv')
    df.drop(columns=['split', 'genre', 'dataset', 'year', 'sid'], axis=1, inplace=True)
    df['s1'] = df['sentence1']
    df['s2'] = df['sentence2']
    df.drop(columns=['sentence1', 'sentence2'], axis=1, inplace=True)
    df['score'] = df['score'].astype(float)*0.2 # Scale between 0 and 1
    df = df[['s1', 's2', 'score']]
    df.insert(0, 'id', [f'sts_{dataset_type}_' + str(i) for i in range(1, len(df) + 1)])
    df.to_csv(f'./datasets/final/sts-{dataset_type}.csv', index=False, header=True)

# Quora Question Pairs Dataset

## Transform

In [64]:
df = pd.read_csv('./datasets/raw/quora_questions.csv')
df.drop(columns=['id', 'qid1', 'qid2'], axis=1, inplace=True)
df['s1'] = df['question1']
df['s2'] = df['question2']
df.drop(columns=['question1', 'question2'], axis=1, inplace=True)
df['score'] = df['is_duplicate'].astype(float)
df = df[['s1', 's2', 'score']]

dataframes = { 'train': None, 'test': None, 'val': None}

dataframes['train'], dataframes['test'] = train_test_split(df, test_size=0.2, random_state=42)
dataframes['train'], dataframes['val'] = train_test_split(dataframes['train'], test_size=0.1, random_state=42)
for dataset_type in ['train', 'test', 'val']:
    dataframes[dataset_type].insert(0, 'id', [f'quora_{dataset_type}_' + str(i) for i in range(1, len(dataframes[dataset_type]) + 1)])
    dataframes[dataset_type].to_csv(f'./datasets/final/quora-{dataset_type}.csv', index=False, header=True)

# SICK Dataset

## Transform

In [65]:
import pandas as pd

df = pd.read_csv('./datasets/raw/SICK.txt', sep='\t')

df = df.rename(columns={
    # 'pair_ID': 'id',
    'sentence_A': 's1',
    'sentence_B': 's2',
    'relatedness_score': 'score',
})

df['score'] = df['score'] / 5.0

unified_columns = [
    's1', 's2', 'score'
]

df = df[unified_columns]

dataframes = { 'train': None, 'test': None, 'val': None}

dataframes['train'], dataframes['test'] = train_test_split(df, test_size=0.2, random_state=42)
dataframes['train'], dataframes['val'] = train_test_split(dataframes['train'], test_size=0.1, random_state=42)
for dataset_type in ['train', 'test', 'val']:
    dataframes[dataset_type].insert(0, 'id', [f'sick_{dataset_type}_' + str(i) for i in range(1, len(dataframes[dataset_type]) + 1)])
    dataframes[dataset_type].to_csv(f'./datasets/final/sick-{dataset_type}.csv', index=False, header=True)

# MSR

## Transform

In [66]:
def load_ms_paraphrase(filepath):
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        lines = lines[1:]
        for line in lines:
            if not line.strip() or line.startswith('//'):
                continue
                
            parts = line.strip().split('\t')
            if len(parts) >= 5:
                quality = parts[0]
                id1 = parts[1]
                id2 = parts[2]
                string1 = '\t'.join(parts[3:-1])
                string2 = parts[-1]
                data.append([quality, id1, id2, string1, string2])
    return pd.DataFrame(data, columns=['Quality', '#1 ID', '#2 ID', '#1 String', '#2 String'])


def transform_msr(df, dataset_type):
    df.drop(columns=['#1 ID', '#2 ID'], axis=1, inplace=True)
    df['s1'] = df['#1 String']
    df['s2'] = df['#2 String']
    df.drop(columns=['#1 String', '#2 String'], axis=1, inplace=True)
    df['score'] = df['Quality'].astype(float)
    df = df[['s1', 's2', 'score']]
    df.insert(0, 'id', [f'msr_{dataset_type}_' + str(i) for i in range(1, len(df) + 1)])
    df.to_csv(f'./datasets/final/msr-{dataset_type}.csv', index=False, header=True)

train_df = load_ms_paraphrase('datasets/raw/msr_train.txt')
test_df = load_ms_paraphrase('datasets/raw/msr_test.txt')

train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)


transform_msr(train_df, 'train')
transform_msr(test_df, 'test')
transform_msr(val_df, 'val')

# Data summary

In [67]:
datasets = [
    'datasets/final/sts-train.csv',
    'datasets/final/sts-test.csv',
    'datasets/final/sts-val.csv',
    'datasets/final/quora-train.csv',
    'datasets/final/quora-test.csv',
    'datasets/final/quora-val.csv',
    'datasets/final/msr-train.csv',
    'datasets/final/msr-test.csv',
    'datasets/final/msr-val.csv',
    'datasets/final/sick-train.csv',
    'datasets/final/sick-test.csv',
    'datasets/final/sick-val.csv',
]

for dataset in datasets:
    df = pd.read_csv(dataset)
    print(dataset, '\t:\t', df.__len__())

datasets/final/sts-train.csv 	:	 5749
datasets/final/sts-test.csv 	:	 1379
datasets/final/sts-val.csv 	:	 1500
datasets/final/quora-train.csv 	:	 291132
datasets/final/quora-test.csv 	:	 80871
datasets/final/quora-val.csv 	:	 32348
datasets/final/msr-train.csv 	:	 3260
datasets/final/msr-test.csv 	:	 1725
datasets/final/msr-val.csv 	:	 816
datasets/final/sick-train.csv 	:	 7084
datasets/final/sick-test.csv 	:	 1968
datasets/final/sick-val.csv 	:	 788


In [3]:
def merge_datasets(dataset1_path, dataset2_path, output_path):
    df1 = pd.read_csv(dataset1_path)
    df2 = pd.read_csv(dataset2_path)
    df = pd.concat([df1, df2], ignore_index=True)
    ## Shuffle the datasets
    df = df.sample(frac=1).reset_index(drop=True)
    df.to_csv(output_path, index=False, header=True)
    
merge_datasets('datasets/final/sts-train.csv', 'datasets/final/sick-train.csv', 'train.csv')
merge_datasets('datasets/final/sts-test.csv', 'datasets/final/sick-test.csv', 'test.csv')
merge_datasets('datasets/final/sts-val.csv', 'datasets/final/sick-val.csv', 'val.csv')